# Exemplo da aplicação (indevida) de Deconvolução de Euler 2D para estimar a profundidade do embasamento. Neste exemplo, o relevo do embasamento é relativamente simples e não apresenta nenhuma falha.

Este [IPython Notebook](http://ipython.org/videos.html#the-ipython-notebook) utiliza a biblioteca de código aberto [Fatiando a Terra](http://fatiando.org/)

In [1]:
#%matplotlib inline
import numpy as np
from scipy import interpolate

from fatiando import gravmag, mesher, utils, gridder
from fatiando.gravmag import prism
from fatiando.mesher import Prism, Polygon
from fatiando.utils import ang2vec, si2nt
from fatiando.gridder import regular
from fatiando.vis import mpl

Na Deconvolução de Euler 2D, as coordenadas $\color{blue}{y_{0}}$ e $\color{blue}{z_{0}}$ da fonte, bem como o nível de base $\color{green}{B}$ são estimados via solução de um sistema linear. Cada linha deste sistema linear e dada por:

$\color{blue}{y_{0}}\dfrac{\partial \Delta T_{i}}{\partial y} + 
 \color{blue}{z_{0}}\dfrac{\partial \Delta T_{i}}{\partial z} +
 \eta \, \color{green}{B} = 
 y_{i}\dfrac{\partial \Delta T_{i}}{\partial y} + 
 z_{i}\dfrac{\partial \Delta T_{i}}{\partial z} +
 \eta \, \Delta T_{i}$ ,

em que $y_{i}$ e $z_{i}$ são as coordenadas do $i$-ésimo ponto de observação, $\Delta T_{i}$, $\dfrac{\partial \Delta T_{i}}{\partial y}$ e $\dfrac{\partial \Delta T_{i}}{\partial z}$ são, respectivamente, a anomalia de campo total (ACT) e suas derivadas espaciais no $i$-ésimo ponto de observação e $\eta$ é um escalar positivo denominado **índice estrutural**. Este parâmetro define o formato da fonte cuja posição será estimada via Deconvolução de Euler. Uma superfície ondulada, tal como o relevo do embasamento sob uma bacia sedimentar, **NÃO** é uma fonte cuja posição pode ser estimada via Deconvolução de Euler. 

In [2]:
def Euler2D(tf, Dytf, Dztf, y, z, si, npontos):
    '''
    Deconvolução de Euler 2D
    
    sistema de coordenadas
      x: dimensão infinita
      y: direção ao longo do perfil
      z: aponta para baixo
    
    input
      tf: array com a anomalia de campo total (ACT)
      Dytf: array com a derivada espacial da ACT na direção y
      Dztf: array com a derivada espacial da ACT na direção z
      y: array com a coordenadas das observações ao longo do perfil
      si: índice estrutural
      npontos: número de pontos em cada janela
      
    output
      y0: array com as coordenadas y estimadas para cada janela
          de dados
      z0: array com as coordenadas z estimadas para cada janela
          de dados
      b: array com os níveis de base estimados para cada janela
          de dados

    '''

    y0 = np.zeros(len(y) + 1 - npontos)
    z0 = np.zeros(len(y) + 1 - npontos)
    b = np.zeros(len(y) + 1 - npontos)
    
    vsi = si*np.ones(npontos)

    # Janela móvel
    for i in range(len(y) + 1 - npontos):
        S = np.vstack((Dytf[i:i+npontos],Dztf[i:i+npontos],vsi)).T
        s = Dytf[i:i+npontos]*y[i:i+npontos] + \
            Dztf[i:i+npontos]*z[i:i+npontos] + \
            vsi*tf[i:i+npontos]
        STS = S.T.dot(S)
        STs = S.T.dot(s)
        # Coordenadas estimadas
        p = np.linalg.solve(STS,STs)
        y0[i] = p[0]
        z0[i] = p[1]
        b[i]  = p[2]

    return y0, z0, b

## Coordenadas das observações e do modelo

In [3]:
ymin = 0.0
ymax = 100000.0
ny = 201
dy = (ymax - ymin)/(ny - 1)
L = ymax - ymin
y = np.linspace(ymin, ymax, ny)
x = np.zeros_like(y)
z = np.zeros_like(y)-150.0

zmin = -2000.0
zmax = 8000.0
xmin = -100000.0
xmax =  100000.0

area = [ymin, ymax, zmax, zmin]

inc, dec = -39.8, -22.5 # inclinação e declinação do campo geomagnético

## Bacia

In [5]:
poligono_bacia = []
bacia = []

# Embasamento inicial
mpl.close('all')
axes = mpl.pyplot.figure().add_subplot(1,1,1)
mpl.ylim(zmax, zmin)
mpl.xlim(ymin, ymax)
mpl.xlabel('y (m)')
mpl.ylabel('z (m)')
mpl.paths([[ymin, 0.0]], [[ymax, 0.0]], style='-k', linewidth=1)
d = mpl.draw_polygon(area, axes, color='r')

In [ ]:
#d = np.array([[  9.97983871e+04,   4.49790795e+01],
#              [  9.33467742e+04,   3.58786611e+02],
#              [  8.83064516e+04,   9.07949791e+02],
#              [  8.30645161e+04,   1.82322176e+03],
#              [  7.78225806e+04,   2.55543933e+03],
#              [  7.03629032e+04,   3.10460251e+03],
#              [  6.33064516e+04,   3.31380753e+03],
#              [  3.32661290e+04,   3.41841004e+03],
#              [  2.74193548e+04,   3.20920502e+03],
#              [  2.09677419e+04,   2.63389121e+03],
#              [  1.67338710e+04,   1.74476987e+03],
#              [  1.37096774e+04,   9.07949791e+02],
#              [  9.47580645e+03,   3.58786611e+02],
#              [  1.20967742e+03,   4.49790795e+01]])

In [6]:
# Embasamento interpolado
f = interpolate.interp1d(d[:,0], d[:,1], kind = 'linear', bounds_error = False, fill_value = 0.0)
nb = 201
by = np.linspace(ymin, ymax, nb)
bz = f(by)
bz[0] = 0.0
bz[nb-1] = 0.0

poligono_bacia = mesher.Polygon(np.vstack((by, bz)).T)

# Visualização 2D da bacia
mpl.figure(figsize=(12,3))
mpl.title('Bacia sedimentar', fontsize=20)
mpl.paths([[ymin, 0.0]], [[ymax, 0.0]], style='-k', linewidth=1)
mpl.polygon(poligono_bacia, style='--k', linewidth=1)
mpl.ylim(zmax, zmin)
mpl.xlabel('y (km)', fontsize=18)
mpl.ylabel('z (km)', fontsize=18)
mpl.xticks(fontsize=16)
mpl.yticks(fontsize=16)
mpl.m2km()
mpl.show()

# Lista de prismas que formam a bacia
db = (ymax - ymin)/(nb - 1)
for i in range(nb):
    bacia.append(mesher.Prism(xmin, xmax, 
                              by[i], by[i] + db,
                              0.0, bz[i], {'magnetization': utils.ang2vec(-0.1, inc, dec)}))

## Anomalia de campo total (ACT)

In [ ]:
tf = gravmag.prism.tf(x, y, z, bacia, inc, dec)

## Visualização da anomalia de campo total

In [ ]:
mpl.close('all')
mpl.figure(figsize=(12,6))

lambida = 0.10

mpl.subplot(2,1,1)
mpl.plot(0.001*y, tf, 'r-')
mpl.plot(0.001*y, tf, 'ro', markersize=3)
vmax = np.max(tf)
vmin = np.min(tf)
dv = lambida*(vmax - vmin)
mpl.ylim(vmin - dv, vmax + dv)
mpl.xlim(0.001*ymin, 0.001*ymax)
mpl.ylabel('ACT (nT)', fontsize=18)
mpl.xticks(fontsize=16)
mpl.yticks(fontsize=16)

mpl.subplot(2,1,2)
mpl.paths([[ymin, 0.0]], [[ymax, 0.0]], style='-k', linewidth=1)
mpl.polygon(poligono_bacia, style='--k', linewidth=1)
dv = lambida*(zmax - zmin)
mpl.ylim(zmax, zmin - dv)
mpl.xlim(ymin, ymax)
mpl.xlabel('y (km)', fontsize=18)
mpl.ylabel('z (km)', fontsize=18)
mpl.xticks(fontsize=16)
mpl.yticks(fontsize=16)
mpl.m2km()
#mpl.savefig('bacia-ACT.png', dpi=600)
mpl.show()

## Derivadas espaciais da ACT

In [ ]:
delta = 10.0

# Derivadas numéricas
Dytf  = gravmag.prism.tf(x, y + delta, z, bacia, inc, dec)
Dytf -= gravmag.prism.tf(x, y - delta, z, bacia, inc, dec)
Dytf  = Dytf/(2.0*delta) # nT/km

Dztf  = gravmag.prism.tf(x, y, z + delta, bacia, inc, dec)
Dztf -= gravmag.prism.tf(x, y, z - delta, bacia, inc, dec)
Dztf  = Dztf/(2.0*delta) # nT/km

# Visualização da anomalia de campo total + derivadas em y e z
mpl.close('all')
mpl.figure(figsize=(12,9))

lambida = 0.10

mpl.subplot(3,1,1)
mpl.paths([[0.001*ymin, 0.0]], [[0.001*ymax, 0.0]], style='-k', linewidth=1.0)
mpl.plot(0.001*y, tf, 'k-')
mpl.plot(0.001*y, tf, 'ko', markersize=3)
vmax = np.max(tf)
vmin = np.min(tf)
dv = lambida*(vmax - vmin)
mpl.ylim(vmin - dv, vmax + dv)
mpl.xlim(0.001*ymin, 0.001*ymax)
mpl.ylabel('ACT (nT)', fontsize=18)
mpl.xticks(fontsize=16)
mpl.yticks(fontsize=16)

mpl.subplot(3,1,2)
mpl.paths([[0.001*ymin, 0.0]], [[0.001*ymax, 0.0]], style='-k', linewidth=1.0)
mpl.plot(0.001*y, 1000.0*Dytf, 'k-')
mpl.plot(0.001*y, 1000.0*Dytf, 'ko', markersize=3)
vmax = np.max(1000.0*Dytf)
vmin = np.min(1000.0*Dytf)
dv = lambida*(vmax - vmin)
mpl.ylim(vmin - dv, vmax + dv)
mpl.xlim(0.001*ymin, 0.001*ymax)
mpl.ylabel('dACT/dy (nT/km)', fontsize=18)
mpl.xticks(fontsize=16)
mpl.yticks(fontsize=16)

mpl.subplot(3,1,3)
mpl.paths([[0.001*ymin, 0.0]], [[0.001*ymax, 0.0]], style='-k', linewidth=1.0)
mpl.plot(0.001*y, 1000.0*Dztf, 'k-')
mpl.plot(0.001*y, 1000.0*Dztf, 'ko', markersize=3)
vmax = np.max(1000.0*Dztf)
vmin = np.min(1000.0*Dztf)
dv = lambida*(vmax - vmin)
mpl.ylim(vmin - dv, vmax + dv)
mpl.xlim(0.001*ymin,0.001* ymax)
mpl.ylabel('dACT/dz (nT/km)', fontsize=18)
mpl.xlabel('y (km)', fontsize=18)
mpl.xticks(fontsize=16)
mpl.yticks(fontsize=16)

mpl.show()

## Parâmetros da Deconvolução de Euler

In [ ]:
npontos = 10 # número de pontos por janela
#npontos = ny # apenas uma janela

dymedio = 0.5*(y[npontos-1] - y[0])

ymedio = np.linspace(ymin + dymedio, 
                     ymax - dymedio, 
                     len(y) + 1 - npontos)

n = [0.5,1,1.5,2,3] # lista de índices estruturais
colors = ['ro','go','bo','yo','ko']
labels = ['0.5','1','1.5','2','3']

mpl.close('all')
mpl.figure(figsize=(12,6))

lambida = 0.10

mpl.subplot(2,1,1)
mpl.plot(0.001*y, tf, 'r-')
mpl.plot(0.001*y, tf, 'ro', markersize=3)
mpl.plot(0.001*y[:npontos], tf[:npontos], 'b-', markersize=5)
mpl.plot(0.001*y[:npontos], tf[:npontos], 'bo', markersize=5)
vmax = np.max(tf)
vmin = np.min(tf)
dv = lambida*(vmax - vmin)
mpl.ylim(vmin - dv, vmax + dv)
mpl.xlim(0.001*ymin, 0.001*ymax)
mpl.ylabel('ACT (nT)', fontsize=18)
mpl.xticks(fontsize=16)
mpl.yticks(fontsize=16)

mpl.subplot(2,1,2)
mpl.paths([[ymin, 0.0]], [[ymax, 0.0]], style='-k', linewidth=1)
mpl.polygon(poligono_bacia, style='--k', linewidth=1)
dv = lambida*(zmax - zmin)
mpl.ylim(zmax, zmin - dv)
mpl.xlim(ymin, ymax)
mpl.xlabel('y (km)', fontsize=18)
mpl.ylabel('z (km)', fontsize=18)
mpl.xticks(fontsize=16)
mpl.yticks(fontsize=16)
mpl.m2km()
#mpl.savefig('bacia-ACT-janela.png', dpi=600)
mpl.show()

## Deconvolução de Euler

In [ ]:
estimativas_y0 = []
estimativas_z0 = []
estimativas_b  = []

for j in range(len(n)):
    
    y0, z0, b = Euler2D(tf, Dytf, Dztf, y, z, n[j], npontos)
    estimativas_y0.append(y0)
    estimativas_z0.append(z0)
    estimativas_b.append(b)

## Visualização das estimativas + ACT

In [ ]:
lambida = 0.10 # parâmetros que controla os eixos

### Todas juntas

In [ ]:
mpl.close('all')
mpl.figure(figsize=(12,6))

mpl.subplot(2,1,1)
mpl.paths([[0.001*ymin, 0.0]], [[0.001*ymax, 0.0]], style='-k', linewidth=1.0)
mpl.plot(0.001*y, tf, 'r-')
mpl.plot(0.001*y, tf, 'ro', markersize=3)
mpl.plot(0.001*y[:npontos], tf[:npontos], 'b-')
mpl.plot(0.001*y[:npontos], tf[:npontos], 'bo', markersize=5)
vmax = np.max(tf)
vmin = np.min(tf)
dv = lambida*(vmax - vmin)
mpl.ylim(vmin - dv, vmax + dv)
mpl.xlim(0.001*ymin, 0.001*ymax)
mpl.ylabel('ACT (nT)', fontsize=18)
mpl.xticks(fontsize=16)
mpl.yticks(fontsize=16)

mpl.subplot(2,1,2)
mpl.paths([[ymin, 0.0]], [[ymax, 0.0]], style='-k', linewidth=1)
mpl.polygon(poligono_bacia, style='--k', linewidth=1)
for j in range(len(n)):
    mpl.plot(estimativas_y0[j], estimativas_z0[j], colors[j], label=labels[j], markersize=7)
dv = lambida*(zmax - zmin)
mpl.ylim(zmax, zmin - dv)
mpl.xlim(ymin, ymax)
mpl.xlabel('y (km)', fontsize=18)
mpl.ylabel('z (km)', fontsize=18)
mpl.xticks(fontsize=16)
mpl.yticks(fontsize=16)
mpl.m2km()
#mpl.savefig('bacia-ACT-estimativas.png', dpi=600)
mpl.show()

### Separadas

In [ ]:
for j in range(len(n)):
    mpl.close('all')
    mpl.figure(figsize=(12,6))
    mpl.subplot(2,1,1)
    mpl.title('$\eta$ = %.1f' % n[j], fontsize=20)
    mpl.paths([[0.001*ymin, 0.0]], [[0.001*ymax, 0.0]], style='-k', linewidth=1.0)
    mpl.plot(0.001*y, tf, 'r-')
    mpl.plot(0.001*y, tf, 'ro', markersize=3)
    mpl.plot(0.001*y[:npontos], tf[:npontos], 'b-')
    mpl.plot(0.001*y[:npontos], tf[:npontos], 'bo', markersize=5)
    vmax = np.max(tf)
    vmin = np.min(tf)
    dv = lambida*(vmax - vmin)
    mpl.ylim(vmin - dv, vmax + dv)
    mpl.xlim(0.001*ymin, 0.001*ymax)
    mpl.ylabel('ACT (nT)', fontsize=18)
    mpl.xticks(fontsize=16)
    mpl.yticks(fontsize=16)

    mpl.subplot(2,1,2)
    mpl.paths([[ymin, 0.0]], [[ymax, 0.0]], style='-k', linewidth=1)
    mpl.polygon(poligono_bacia, style='--k', linewidth=1)
    mpl.plot(estimativas_y0[j], estimativas_z0[j], colors[j], label=labels[j], markersize=7)
    dv = lambida*(zmax - zmin)
    mpl.ylim(zmax, zmin - dv)
    mpl.xlim(ymin, ymax)
    mpl.xlabel('y (km)', fontsize=18)
    mpl.ylabel('z (km)', fontsize=18)
    mpl.xticks(fontsize=16)
    mpl.yticks(fontsize=16)
    mpl.m2km()
    #mpl.savefig('bacia-ACT-est-ien%d.png' % j, dpi=600)
    mpl.show()

## Visualização das estimativas

In [ ]:
lambida = 0.10 # parâmetros que controla os eixos

### Separadas

In [ ]:
for j in range(len(n)):

    mpl.close('all')
    mpl.figure(figsize=(12,3))
    mpl.title('$\eta$ = %.1f' % n[j], fontsize=20)
    mpl.paths([[ymin, 0.0]], [[ymax, 0.0]], style='-k', linewidth=1)
    mpl.polygon(poligono_bacia, style='--k', linewidth=1)
    mpl.plot(estimativas_y0[j], estimativas_z0[j], colors[j], label=labels[j], markersize=7)
    dv = lambida*(zmax - zmin)
    mpl.ylim(zmax, zmin - dv)
    mpl.xlim(ymin, ymax)
    mpl.xlabel('y (km)', fontsize=18)
    mpl.ylabel('z (km)', fontsize=18)
    mpl.xticks(fontsize=16)
    mpl.yticks(fontsize=16)
    mpl.m2km()
    #mpl.savefig('est-ien%d.png' % j, dpi=600)
    mpl.show()

### Todas juntas

In [ ]:
mpl.close('all')
mpl.figure(figsize=(12,3))

mpl.paths([[ymin, 0.0]], [[ymax, 0.0]], style='-k', linewidth=1)
mpl.polygon(poligono_bacia, style='--k', linewidth=1)
for j in range(len(n)):
    mpl.plot(estimativas_y0[j], estimativas_z0[j], colors[j], label=labels[j], markersize=7)
dv = lambida*(zmax - zmin)
mpl.ylim(zmax, zmin - dv)
mpl.xlim(ymin, ymax)
mpl.xlabel('y (km)', fontsize=18)
mpl.ylabel('z (km)', fontsize=18)
mpl.xticks(fontsize=16)
mpl.yticks(fontsize=16)
mpl.m2km()
#mpl.savefig('estimativas.png', dpi=600)
mpl.show()